<a href="https://colab.research.google.com/github/shyamjith94/Machine-Learning-Regression/blob/main/LassoRegressionCorordianteDescent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install turicreate

In [2]:
import turicreate as tc
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from math import log, sqrt
from sklearn.utils import shuffle

In [3]:
data = tc.SFrame.read_csv("/content/drive/My Drive/Machine Learning: Regression/Data/2/c_kc_house_data.csv/kc_house_data.csv")

Finished parsing file /content/drive/My Drive/Machine Learning: Regression/Data/2/c_kc_house_data.csv/kc_house_data.csv

Parsing completed. Parsed 100 lines in 0.996858 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,float,int,float,int,int,float,int,int,int,int,int,int,int,int,int,float,float,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /content/drive/My Drive/Machine Learning: Regression/Data/2/c_kc_house_data.csv/kc_house_data.csv

Parsing completed. Parsed 21613 lines in 0.103685 secs.

In [4]:
def get_numpy_data(frame, features, output):
    """ frame   :- DataFrame
        features:- list of features
        output  :- str output variable(price)
        return  :- numpy array of feature and output
        """
    frame['constant'] = 1
    features = ['constant'] + features
    filter_frame = frame[features]
    # convert to numpy
    feature_matrix = filter_frame.to_numpy()
    output_array = frame[output]
    convert to numpy
    output_matrix = output_array.to_numpy()
    # return
    return feature_matrix, output_matrix

In [5]:
def predict_feature(feature_matrix, weight):
    """ feature_matrix  :- numpy array of features
        weight          :- value of weight to calculate dot product
                           using numpy
        """
    prediction = np.dot(feature_matrix, weight)
    return prediction


In [6]:
def normalize_feature(feature_matrix):
    """ feature_matrix  :- numpy matrix of feature values
        return          :- feature_matrix, normalized feture_matrix 
        """
    norm_feature = np.linalg.norm(feature_matrix, axis=0)
    normalize_feature = feature_matrix/norm_feature
    return normalize_feature,norm_feature 

**Implementing Coordinate Descent with normalized features**
**SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).**

In [9]:
simple_features = ["sqft_living", "bedrooms"]
my_output = "price"
simple_feature_matrix, output = get_numpy_data(data, simple_features, my_output) 
print("Features")
print(simple_feature_matrix[:10])
print("\nOutput")
print(output[:10])

Features
[[   1 1180    3]
 [   1 2570    3]
 [   1  770    2]
 [   1 1960    4]
 [   1 1680    3]
 [   1 5420    4]
 [   1 1715    3]
 [   1 1060    3]
 [   1 1780    3]
 [   1 1890    3]]

Output
[ 221900.  538000.  180000.  604000.  510000. 1225000.  257500.  291850.
  229500.  323000.]


In [16]:
simple_feature_normalize, normlize = normalize_feature(simple_feature_matrix)
print("Normalize Features")
print(simple_feature_normalize[:10])
print("\nNorm")
print(normlize[:10])

Normalize Features
[[0.00680209 0.00353021 0.00583571]
 [0.00680209 0.00768869 0.00583571]
 [0.00680209 0.00230361 0.00389048]
 [0.00680209 0.00586375 0.00778095]
 [0.00680209 0.00502607 0.00583571]
 [0.00680209 0.01621506 0.00778095]
 [0.00680209 0.00513078 0.00583571]
 [0.00680209 0.00317121 0.00583571]
 [0.00680209 0.00532524 0.00583571]
 [0.00680209 0.00565433 0.00583571]]

Norm
[1.47013605e+02 3.34257264e+05 5.14075870e+02]


In [23]:
my_weight = np.array([1.,4.,1.])
prediction = predict_feature(simple_feature_normalize, my_weight )
prediction

array([0.02675867, 0.04339256, 0.01990703, ..., 0.02289873, 0.03178473,
       0.02289873])

```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```

**ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].**<br>

**Compute the values of ro[i] for each feature in this simple model, using the formula given above, using the formula:**

In [28]:
ro = np.array([0,0,0])
ro[1] = np.sum(simple_feature_normalize[:,1]* (output - prediction + weight[1]*simple_feature_normalize[:,1]))
ro[2] = np.sum(simple_feature_normalize[:,2]* (output - prediction + weight[2]*simple_feature_normalize[:,2]))
print(ro)

[       0 87939470 80966698]


**The range of values of `l1_penalty` would not set `w[1]` zero, but would set `w[2]` to zero**


In [30]:
print("ranges would NOT set w[1] to zero :\t", (-ro[1]*2, ro[1]*2))
print("ranges would NOT set w[2] to zero :\t", (-ro[2]*2, ro[2]*2))

print("ranges of l1_penalty set w2 zero but NOT set w1 zero:\t", (-ro[1]*2, -ro[2]*2), (ro[2]*2, ro[1]*2))

ranges would NOT set w[1] to zero :	 (-175878940, 175878940)
ranges would NOT set w[2] to zero :	 (-161933396, 161933396)
ranges of l1_penalty set w2 zero but NOT set w1 zero:	 (-175878940, -161933396) (161933396, 175878940)


**Single Coordinate Descent Step**

In [46]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    """ First we compute prediction and find the ro[i] and check ther regurlize or not
        i               :- weights
        featrure_matrix :- normalized feature matrix
        output          :- ourcase its price should string value
        l1_penalty      :- l1 regulization penalty value
        """
    # compute prediction
    prediction = predict_feature(feature_matrix, weights)
    # compute ro[i]
    ro_i = np.sum(feature_matrix[:,i] * (output - prediction + weights[i] * feature_matrix[:,i]))
    if i == 0:
        new_weight_i = ro_i
    elif ro_i < -l1_penalty /2.:
        new_weight_i = (ro_i + l1_penalty/2)
    elif ro_i > l1_penalty/2.:
        new_weight_i = (ro_i - l1_penalty/2)
    else:
        new_weight_i = 0.
    return new_weight_i 

**Cyclical coordinate descent**

In [43]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    """ Each time we scan all the coordinates (features) once, we measure the change in weight 
        for each coordinate. If no coordinate changes by more than a specified threshold, we stop.
        
        featrure_matrix :- normalized feature matrix
        output          :- ourcase its price should string value
        l1_penalty      :- l1 regulization penalty value
        """
    weights = initial_weights
    max_coordinate_change = 0.0

    for i in range(len(weights)):
        old_weights_i = weights[i]

        weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
        coordinate_change_i = old_weights_i - weights[i]
        if coordinate_change_i > max_coordinate_change:
            max_coordinate_change = coordinate_change_i
            print(coordinate_change_i)
    if max_coordinate_change <= tolerance:
        return weights
    else:
         lasso_cyclical_coordinate_descent(feature_matrix, output, weights, l1_penalty, tolerance)



**Then, run implementation of LASSO coordinate descent**

In [33]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

In [59]:
(simple_feature_matrix, output) = get_numpy_data(data, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_feature(simple_feature_matrix) # normalize features

In [60]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
weights

0.44476933777332306


array([21624991.05768758, 63157253.52158205,        0.        ])

In [61]:
lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

0.37219709157943726


array([21624991.46455593, 63157253.14938496,        0.        ])

In [63]:
predictions = predict_feature(normalized_simple_feature_matrix, weights)
print("RSS of the learned model on the normalized dataset:\t{:.5e}".format(((output - predictions)**2).sum()))

RSS of the learned model on the normalized dataset:	1.63049e+15


**Evaluating LASSO fit with more features**

In [64]:
train_data,test_data = data.random_split(.8,seed=0)

In [65]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

In [67]:
(all_feature_matrix, output) = get_numpy_data(train_data, all_features, my_output)
(normalized_all_feature_matrix, all_norms) = normalize_feature(all_feature_matrix) # normalize features

In [72]:
initial_weights = np.zeros(14)
l1_penalty = 1e7
tolerance = 1.0

In [74]:
weights1e7 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

0.7191657610237598


In [75]:
print(weights1e7)

[24429599.51523736        0.                0.         48389175.44877578
        0.                0.          3317511.21586376  7329961.76733328
        0.                0.                0.                0.
        0.                0.        ]


In [82]:
feature_list = ['constant'] + all_features

In [83]:
for k,v in zip(feature_list, weights1e7):
    if v != 0.0:
        print(k, v)

constant 24429599.515237365
sqft_living 48389175.44877578
waterfront 3317511.215863756
view 7329961.767333284


In [76]:
initial_weights = np.zeros(14)
l1_penalty = 1e8
tolerance = 1.0

In [78]:
weights1e8 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

In [79]:
weights1e8

array([71114625.71488702,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ])

**Features had non-zero weight in this case**

In [84]:
for k,v in zip(feature_list, weights1e8):
    if v != 0.0:
        print(k, v)

constant 71114625.71488702


**Finally, learn the weights with l1_penalty=1e4, on the training data. Initialize weights to all zeros, and set the tolerance=5e5.**

In [90]:
initial_weights = np.zeros(14)
l1_penalty = 1e4
tolerance = 5e5

In [93]:
weights1e4 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

57911.28881525248
188183.99360296316
496485.9123094268


In [94]:
weights1e4

array([ 77038859.60208486, -21635681.75037373,  15536145.41333864,
        83990080.81003869,  -2017288.60867553,  -5431729.45690593,
         6481705.37853717,   7246557.04962423,   2241631.18760191,
         8531074.43396704,  -7232536.84361754,  -3077425.33358186,
       -76356323.75845608,   2780146.66815214])

In [95]:
for k,v in zip(feature_list, weights1e4):
    if v != 0.0:
        print(k, v)

constant 77038859.60208486
bedrooms -21635681.750373732
bathrooms 15536145.413338635
sqft_living 83990080.81003869
sqft_lot -2017288.6086755306
floors -5431729.456905927
waterfront 6481705.378537168
view 7246557.049624234
condition 2241631.1876019062
grade 8531074.433967035
sqft_above -7232536.843617544
sqft_basement -3077425.3335818555
yr_built -76356323.75845608
yr_renovated 2780146.668152137


In [97]:
my_output = 'price'
(feature_matrix, output) = get_numpy_data(train_data, all_features, my_output)
normalized_feature_matrix, norms = normalize_feature(feature_matrix)

In [98]:
normalized_weights1e7 = weights1e7 / norms
normalized_weights1e8 = weights1e8 / norms
normalized_weights1e4 = weights1e4 / norms
print(normalized_weights1e7[3])

161.3174599038232


In [99]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

In [101]:
prediction = predict_feature(test_feature_matrix, normalized_weights1e4)
rss = ((test_output - prediction)**2).sum()
print('RSS for model with weights1e4 is {:.5e}'.format(rss))

RSS for model with weights1e4 is 2.31990e+14


In [102]:
prediction = predict_feature(test_feature_matrix, normalized_weights1e7)
rss = ((test_output - prediction)**2).sum()
print('RSS for model with weights1e7 is {:.5e}'.format(rss))

RSS for model with weights1e7 is 2.75962e+14


In [103]:
prediction = predict_feature(test_feature_matrix, normalized_weights1e8)
rss = ((test_output - prediction)**2).sum()
print('RSS for model with weights1e8 is {:.5e}'.format(rss))

RSS for model with weights1e8 is 5.37166e+14
